In [1]:
from transformers import pipeline

d:\Udemy\huggingface\analise_sentimento\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
classifier = pipeline("zero-shot-classification",model="facebook/bart-large-mnli")

d:\Udemy\huggingface\analise_sentimento\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [3]:
categories = ['Outdoor Gear', 'Kitchen Appliances', 'Apparel']

In [4]:
phrase1 = "This lightweight backpack is perfect for outdoor adventures and hiking trips"
phrase2 = "This high-end blender features multiple speed settings and a durable stainless steel blade"
phrase3 = "This cozy, hand-knitted sweater is made from 100% organic wool and available is various sizes"

In [5]:
predict = classifier(phrase1, candidate_labels=categories)
predict

{'sequence': 'This lightweight backpack is perfect for outdoor adventures and hiking trips',
 'labels': ['Outdoor Gear', 'Apparel', 'Kitchen Appliances'],
 'scores': [0.9847255349159241, 0.011576280929148197, 0.0036982304882258177]}

In [6]:
predict = classifier(phrase2, candidate_labels=categories)
predict

{'sequence': 'This high-end blender features multiple speed settings and a durable stainless steel blade',
 'labels': ['Kitchen Appliances', 'Outdoor Gear', 'Apparel'],
 'scores': [0.8003118634223938, 0.13100369274616241, 0.0686844140291214]}

In [7]:
predict = classifier(phrase3, candidate_labels=categories)
predict

{'sequence': 'This cozy, hand-knitted sweater is made from 100% organic wool and available is various sizes',
 'labels': ['Apparel', 'Outdoor Gear', 'Kitchen Appliances'],
 'scores': [0.7891675233840942, 0.13525329530239105, 0.0755791887640953]}

In [1]:
from liqfit.pipeline import ZeroShotClassificationPipeline
from liqfit.models import T5ForZeroShotClassification
from transformers import T5Tokenizer

d:\Udemy\huggingface\analise_sentimento\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = T5ForZeroShotClassification.from_pretrained('knowledgator/comprehend_it-multilingual-t5-base')
tokenizer = T5Tokenizer.from_pretrained('knowledgator/comprehend_it-multilingual-t5-base')
classifier = ZeroShotClassificationPipeline(model=model, tokenizer=tokenizer,
                        hypothesis_template = '{}', encoder_decoder = True)

You are using a model of type T5 to instantiate a model of type t5. This is not supported for all configurations of models and can yield errors.
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [3]:
categorias = ["Esporte", "Moda", "Ciência"]
frase1 = "O livro sobre astronomia foi muito informativo e interessante."
frase2 = "Ela comprou um vestido novo para a festa de aniversário."
frase3 = "O time de futebol venceu o campeonato estadual."

In [4]:
resultado = classifier(frase1, categorias, multi_label=False)
resultado

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'sequence': 'O livro sobre astronomia foi muito informativo e interessante.',
 'labels': ['Ciência', 'Esporte', 'Moda'],
 'scores': [0.6034311056137085, 0.26732730865478516, 0.12924163043498993]}

In [5]:
resultado = classifier(frase2, categorias, multi_label=False)
resultado

{'sequence': 'Ela comprou um vestido novo para a festa de aniversário.',
 'labels': ['Moda', 'Esporte', 'Ciência'],
 'scores': [0.6500799655914307, 0.18261080980300903, 0.16730928421020508]}

In [6]:
resultado = classifier(frase3, categorias, multi_label=False)
resultado

{'sequence': 'O time de futebol venceu o campeonato estadual.',
 'labels': ['Esporte', 'Ciência', 'Moda'],
 'scores': [0.8571128249168396, 0.12767237424850464, 0.015214790590107441]}

In [7]:
import pandas as pd

In [8]:
df_frases = pd.read_csv("dados/frases2.csv")
df_frases.head()

,ID,Frase
0,1,O time de futebol venceu o campeonato estadual.
1,2,O novo uniforme da equipe está disponível para...
2,3,A Copa do Mundo de 2026 será realizada nos Est...
3,4,Ela decidiu usar um vestido vermelho brilhante...
4,5,O jogador foi eleito o melhor atleta do ano.


In [11]:
resultado_df = classifier(df_frases["Frase"], categorias, multi_label=False)
resultado_df['labels']
resultado_df['scores']

[0.8571128249168396, 0.12767237424850464, 0.015214790590107441]

In [23]:
def classificar(frase):
    resultado_df = classifier(frase, categorias, multi_label=False)
    if 'scores' not in resultado_df or 'labels' not in resultado_df:
        raise ValueError("Resultado não possui scores ou labels")
    scores = resultado_df["scores"]
    labels = resultado_df["labels"]
    
    if not scores or not labels:
        raise ValueError("Lista de Scores ou labels está vazia")
    
    max_score_index = scores.index(max(scores))
    
    return labels[max_score_index]
    

In [24]:
df_frases["Categoria"] = df_frases["Frase"].apply(classificar)
df_frases

,ID,Frase,Categoria
0,1,O time de futebol venceu o campeonato estadual.,Esporte
1,2,O novo uniforme da equipe está disponível para...,Esporte
2,3,A Copa do Mundo de 2026 será realizada nos Est...,Esporte
3,4,Ela decidiu usar um vestido vermelho brilhante...,Moda
4,5,O jogador foi eleito o melhor atleta do ano.,Esporte
5,6,O desfile de moda na semana passada apresentou...,Moda
6,7,O campeonato de basquete está se aproximando e...,Esporte
7,8,O cientista fez uma descoberta importante sobr...,Ciência
8,9,A última coleção de verão foi inspirada nas co...,Ciência
9,10,O novo estádio será inaugurado no próximo mês.,Ciência
